***
# Nettoyage des données
***

In [1]:
# ====== Importer les library ====== #
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)

In [2]:
# ====== Déterminer les path ====== #
import os
cwd = os.getcwd()
parent = os.path.dirname(cwd)
data_path = parent + '\\data\\'

In [3]:
# ====== Importer les df ====== #
comments = pd.read_csv(data_path + 'Comments.csv')
posts = pd.read_csv(data_path + 'Posts.csv')

comments.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time,id,like_count,message,parent,postId
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,2023-06-02T16:06:59+0000,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,2023-06-02T16:21:43+0000,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613


In [6]:
test = pd.read_csv(data_path + 'Test.csv')
test.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time,id,like_count,message,parent,postId
0,0.000548,0.007893,0.011800,0.000517,0.005419,0.006754,-1,2023-06-01T18:32:37+0000,721844289946959_831368741921536,0,Mon ex🤔?,NaN,192978590727638_721844289946959
1,0.125566,0.469236,0.501357,0.169603,0.012389,0.434752,-1,2023-06-01T18:51:50+0000,721844289946959_9565562033514339,0,"""Quel est ton nom ? Je te l' ordonne de le di...",NaN,192978590727638_721844289946959


In [146]:
# ====== Merge les df ====== #
dftest_merge = pd.merge(test, posts, left_on='postId', right_on='id', indicator=True, suffixes=('_comments', '_posts'))
dftest_merge.head(2)

# *** Un algo qui check si le titre est déjà dans le set de données? 

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time_comments,id_comments,like_count,message_comments,parent,postId,attachments.data,created_time_posts,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title,_merge
0,0.000548,0.007893,0.011800,0.000517,0.005419,0.006754,-1,2023-06-01T18:32:37+0000,721844289946959_831368741921536,0,Mon ex🤔?,NaN,192978590727638_721844289946959,"[{""title"":""Un troupeau de voix enragées"",""unsh...",2023-06-01T18:31:33+0000,192978590727638_721844289946959,chroniques,CHRONIQUE / Entre 4 à 10% des Québécois entend...,https://www.facebook.com/553956616735728/posts...,patrick-duquette,0,Un troupeau de voix enragées,both
1,0.125566,0.469236,0.501357,0.169603,0.012389,0.434752,-1,2023-06-01T18:51:50+0000,721844289946959_9565562033514339,0,"""Quel est ton nom ? Je te l' ordonne de le di...",NaN,192978590727638_721844289946959,"[{""title"":""Un troupeau de voix enragées"",""unsh...",2023-06-01T18:31:33+0000,192978590727638_721844289946959,chroniques,CHRONIQUE / Entre 4 à 10% des Québécois entend...,https://www.facebook.com/553956616735728/posts...,patrick-duquette,0,Un troupeau de voix enragées,both


In [147]:
# ====== Merge les df ====== #
df_merge = pd.merge(comments, posts, left_on='postId', right_on='id', indicator=True, suffixes=('_comments', '_posts'))
df_merge.head(2)


# Analyse du merge 
unique_list = []
for value in df_merge['_merge'].unique():
    unique_list.append(value)

if (len(unique_list) == 1) & (unique_list[0] == 'both'):
    print('Tous les lignes merge')
else :
    print('Certaines lignes ne sont pas merge')


df_merge = df_merge.drop(columns='_merge')

Tous les lignes merge


In [148]:
# Attention au Nan
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_22784\284039997.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['created_time_comments', 'id_comments', 'message_comments', 'parent',
       'postId', 'attachments.data', 'created_time_posts', 'id_posts',
       'mainTopic', 'message_posts', 'permalink_url', 'secondTopic', 'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


***
### Compléter les valeurs manquantes

#### Solution
- Drop parent
- Drop missings title row 
***

In [149]:
df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,created_time_comments,id_comments,like_count,message_comments,parent,postId,attachments.data,created_time_posts,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,2023-06-02T16:06:59+0000,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",2023-06-02T16:03:24+0000,192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,2023-06-02T16:21:43+0000,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",2023-06-02T16:03:24+0000,192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence


In [150]:
# ====== Visualisation des valeurs manquantes ====== #
print('Missing value')
print('*'*25)
print(f'Nombre de lignes : {len(df_merge)}')
for col in df_merge.columns.to_list():
    nb_na = df_merge[col].isna().sum()
    perc = round(df_merge[col].isna().sum()/len(df_merge)*100)
    print(f'{col.ljust(21)} : {str(nb_na).rjust(11)} ({perc}%)')

Missing value
*************************
Nombre de lignes : 935698
IDENTITY_ATTACK       :           0 (0%)
INSULT                :           0 (0%)
PROFANITY             :           0 (0%)
SEVERE_TOXICITY       :           0 (0%)
THREAT                :           0 (0%)
TOXICITY              :           0 (0%)
comment_count         :           0 (0%)
created_time_comments :           0 (0%)
id_comments           :           0 (0%)
like_count            :           0 (0%)
message_comments      :           0 (0%)
parent                :      459661 (49%)
postId                :           0 (0%)
attachments.data      :          35 (0%)
created_time_posts    :           0 (0%)
id_posts              :           0 (0%)
mainTopic             :        8481 (1%)
message_posts         :       10158 (1%)
permalink_url         :           0 (0%)
secondTopic           :       67076 (7%)
shares                :           0 (0%)
title                 :        8457 (1%)


***
## Variable de temps
***

In [151]:
# ====== convertir le temps en format datetime ====== #
df_merge['comment_time'] = pd.to_datetime(df_merge['created_time_comments'])
df_merge['post_time'] = pd.to_datetime(df_merge['created_time_posts'])
df_merge = df_merge.drop(columns=['created_time_comments', 'created_time_posts'])
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,id_comments,like_count,message_comments,parent,postId,attachments.data,id_posts,mainTopic,message_posts,permalink_url,secondTopic,shares,title,comment_time,post_time
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,722477749883613_788168749338745,0,"Faux, ce ne sera jamais le temps pour lui car ...",NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:06:59+00:00,2023-06-02 16:03:24+00:00
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,722477749883613_1593974021087773,1,Mon dieu!! On nous prend sérieusement pour des...,NaN,192978590727638_722477749883613,"[{""title"":""Les incohérences de l’ingérence"",""u...",192978590727638_722477749883613,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,https://www.facebook.com/553956616735728/posts...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:21:43+00:00,2023-06-02 16:03:24+00:00


In [152]:
# ====== Éliminer les colonnes non-pertinantes ====== #
colones_non_utiles = ['id_comments', 'parent', 'postId', 'attachments.data', 'id_posts', 'permalink_url']
df_merge = df_merge.drop(columns=colones_non_utiles)
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,comment_time,post_time
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:06:59+00:00,2023-06-02 16:03:24+00:00
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,2023-06-02 16:21:43+00:00,2023-06-02 16:03:24+00:00


In [153]:
# Attribut différence de temps
df_merge['time_difference'] = df_merge['comment_time'] - df_merge['post_time'] 
df_merge['time_difference'] = df_merge['time_difference'].astype('timedelta64[m]')
df_merge.head(2)

# Attributs temps solo
df_merge['year'] = df_merge['comment_time'].dt.year
df_merge['month'] = df_merge['comment_time'].dt.month
df_merge['weekday'] = df_merge['comment_time'].dt.weekday       # Monday=0, Sunday=6
df_merge = df_merge.drop(columns=['comment_time', 'post_time'])
df_merge.head(2)


,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [154]:
# Attention au Nan
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_22784\284039997.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


***
## Les sujets : Étiquettage
***

In [27]:
df_merge[["title"]]

,title
0,Les incohérences de l’ingérence
1,Legault veut empêcher les antiavortement de fa...
2,S’envoyer en l’air à 97 ans
3,Une famille québécoise vit dans une géonef aut...
4,Influenceurs : gare aux publicités trompeuses
...,...
475371,La CAQ prévoit améliorer les conditions de tra...
475372,La CAQ prévoit améliorer les conditions de tra...
475373,La CAQ prévoit améliorer les conditions de tra...
475374,La CAQ prévoit améliorer les conditions de tra...


***
## Outliers
***

***** Note : Je ne suis pas certain qu'on veuille enlever les outliers, puisque c'est probablement ces observation (avec de grosses valeurs) qui recoivent des commentaires.
Peut-être mettre les valeurs > upperbound = upperbound

In [15]:
df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [35]:
def countNumberOutlier(df, col):
    nbval_initial = len(df[col])
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    nbval_final = len(df.loc[df[col]<upper_bound])
    perte = nbval_initial - nbval_final
    perte_proportion = round(perte/nbval_initial, 2)

    print('*'*25)
    print(f'Colonne : {col}')
    print(f'Nombre d\'observation initial : {nbval_initial}')
    print(f'Nombre d\'observation sans outliers : {nbval_final}')
    print(f'Perte : {perte} ({perte_proportion} %)')

In [36]:
cols = ['IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'SEVERE_TOXICITY', 'THREAT', 'TOXICITY', 'comment_count', 'like_count']
df = df_merge.copy()

for col in cols:
    countNumberOutlier(df, col)

*************************
Colonne : IDENTITY_ATTACK
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 807410
Perte : 128288 (0.14 %)
*************************
Colonne : INSULT
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 812232
Perte : 123466 (0.13 %)
*************************
Colonne : PROFANITY
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 770424
Perte : 165274 (0.18 %)
*************************
Colonne : SEVERE_TOXICITY
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 795753
Perte : 139945 (0.15 %)
*************************
Colonne : THREAT
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 819678
Perte : 116020 (0.12 %)
*************************
Colonne : TOXICITY
Nombre d'observation initial : 935698
Nombre d'observation sans outliers : 869007
Perte : 66691 (0.07 %)
*************************
Colonne : comment_count
Nombre d'observation initial : 935

In [ ]:
# Valeur > upperbound = upperbound
for col in cols:
    df[col] = np.where(test[df] > test[df].quantile(0.75), test[df].quantile(0.75), test[df])

In [ ]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

***
## Transformation
***

In [7]:
#====== Mesure de l'assymétrie (skewness Kurtosis) ======#

# Skewness
#       Skewness positif : La queue de la distribution est étendue du côté droit
#       Skewness négatif : La queue de la distribution est étendue du côté gauche 
#       Skewness nul : La distribution est symétrique par rapport à la moyenne
#               skewness est entre -0.5 et 0.5, la distribution est relativement symétrique.

# Kurtosis
#       Kurtosis élevé (positif) : Les queues de la distribution sont plus lourdes que la distribution normale, et les pics sont plus élevés et plus étroits. Cela indique des valeurs aberrantes plus importantes.
#       Kurtosis faible (négatif) : Les queues de la distribution sont plus légères que la distribution normale, et les pics sont plus plats. Cela indique des valeurs aberrantes moins importantes.
#       Kurtosis standard (zéro) : La distribution a la même forme que la distribution normale (mesurée en termes de pics et de queues).
#               Un kurtosis entre -2 et 2 est considéré comme normal (mesuré par rapport à une distribution normale).
#               Un kurtosis supérieur à 2 indique une distribution plus « lourde » que la normale (plus de valeurs dans les queues).

import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4


In [ ]:
# cols = ['IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'SEVERE_TOXICITY', 'THREAT', 'TOXICITY', 'comment_count', 'like_count']
# df = df_merge[cols].copy()

# for col in cols: 
#     df[col+'_log'] = np.log1p(df[col])
#     df[col+'_cube'] = np.power(df[col], 1/3)
# df.head(2)

In [8]:
cols = ['IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'SEVERE_TOXICITY', 'THREAT', 'TOXICITY', 'comment_count', 'like_count', 'shares']
for col in cols: 
    # df_merge[col+'_log'] = np.log1p(df_merge[col])
    df_merge[col+'_cube'] = np.power(df_merge[col], 1/3)
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0


In [9]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_20500\1801815014.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


In [157]:
def printSkewAndKurtosis(df, col):
    print('*'*25)
    print(f'Colonne : {col}')
    print('moyenne : ', df[col].mean())
    print('mediane : ', df[col].median())
    print('skewness : ', round(skew(df[col])))
    print('Kurtosis :', round(kurtosis(df[col])))

def printHistogram(df, col):
    plt.hist(df[col])
    plt.title('comment_count')
    plt.ylabel('Fréquence')
    plt.xlabel('Value')
    plt.show()

def generateStatsValues(df, col):
    moyenne = round(df[col].mean(), 2)
    mediane = round(df[col].median(), 2)
    skew_value = round(skew(df[col]), 2)
    kurtosis_value = round(kurtosis(df[col]), 2)
    return moyenne, mediane, skew_value, kurtosis_value

In [158]:
stats_dict = {}

cols_to_dict = df_merge.columns.to_list() 

for col in cols_to_dict:
    stats_dict[col] = generateStatsValues(df_merge, col)

stats_df = pd.DataFrame(stats_dict).transpose().rename(columns={0:'Moyenne', 1:'Mediane', 2:'Skewness', 3:'Kurtosis'}).sort_index()
stats_df

# Rappel : 
# Skewness est entre -0.5 et 0.5
# Kurtosis entre -2 et 2 est considéré comme normal

***
## Extraction des mots dans les publications
***

1. Mots pertinent (les titres et les commentaires, les commentaires parents)
2. Les émoji

In [10]:
df_merge =pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative,emoji_count
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,Faux jamais temps réélu parti crève faim Jamai...,[],[],0.077610,0.130838,0.791552,0
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots,[],[],0.697028,0.047872,0.255100,0


In [5]:
# df_test = df_merge.head(2000).copy()

In [58]:
# Nettoyer stop words avec spacy
import spacy
import concurrent.futures
import re

nlp = spacy.load('fr_core_news_sm')

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def cleanTextWithSpacy(text):
    text = remove_emojis(text)
    doc = nlp(text)
    clean_tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    clean_text = ' '.join(clean_tokens)
    clean_text = clean_text.replace('\n', ' ').replace('  ', ' ').strip().replace('¿', '').replace('¡', '')
    clean_text = clean_text.replace(u'«', '').replace(u'»', '').replace("'", '').replace("-", ' ')
    return clean_text


def applyCleaningText(df, initial_col_name, final_col_name):
    df[final_col_name] = df[initial_col_name].apply(lambda x: cleanTextWithSpacy(x))
    return df


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    results = list(executor.map(applyCleaningText, [df_merge], ['message_comments'], ['messages_clean']))

# df_test = applyCleaningText(df_test, 'message_comments', 'messages_clean')
df_merge.head(5)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,Faux jamais temps réélu parti crève faim Jamai...,[],[],0.077610,0.130838,0.791552
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100
2,0.029144,0.269578,0.016753,0.006104,0.005864,0.246033,0,0,"Denise Morais ..Oui et avec raison ! (Car ""nou...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,86.0,2023,6,4,0.307741,0.645994,0.255876,0.182751,0.180331,0.626611,0.0,0.0,0.0,Denise Morais Oui raison majorité tyrannique ...,[],[],0.243069,0.089332,0.667599
3,0.036105,0.402434,0.479813,0.023411,0.008220,0.372227,0,0,Gros show. Mais pendant la plandémie chinoise ...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,94.0,2023,6,4,0.330514,0.738298,0.782872,0.286072,0.201820,0.719343,0.0,0.0,0.0,Gros show plandémie chinoise tlm fermait bien ...,[],[],0.685404,0.125043,0.189553
4,0.280349,0.520819,0.163573,0.169603,0.008790,0.439655,0,0,On apellait ça chinada pis les vaxxinés compre...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,95.0,2023,6,4,0.654485,0.804567,0.546895,0.553534,0.206378,0.760391,0.0,0.0,0.0,apellait chinada pis vaxxinés comprenaient rie...,[],[],0.158446,0.242306,0.599248


In [61]:
# df_test[['message_comments', 'messages_clean']].to_excel('message.xlsx')

In [11]:
# Extraire les Émoji
import re
import emoji

def findEmoji(text):
    emojis = re.findall("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", text)
    return emojis


##### Créer une liste de valeur plutot qu'un bloc de text #####
def emojiToText(text):
    emojiInText = emoji.demojize(text)
    emojiInText = emojiInText[1:-1]
    return emojiInText

df_merge['emojis'] = df_merge['message_comments'].apply(lambda x: findEmoji(x))
df_merge['emojis_text'] = df_merge['emojis'].apply(lambda x:[emojiToText(value) for value in x])



df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative,emoji_count
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,Faux jamais temps réélu parti crève faim Jamai...,[],[],0.077610,0.130838,0.791552,0
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100,0


In [12]:
# Attribut nombre d'émoji
df_merge['emoji_count'] = df_merge['emojis_text'].apply(lambda x: len(x))

In [53]:
# Garder uniquement les emoji dont les observations on des commentaires
df_commented = df_merge[['emojis_text', 'comment_count']].copy()
df_commented = df_commented.loc[df_commented['comment_count']>0]


# Retreive unique emoji from 
def createEmojiList(list_element, emoji_list):
    for element in list_element:
        if element == '':
            pass
        elif element not in emoji_list:
            emoji_list.append(element)
    return emoji_list

emoji_list = []

df_commented['emojis_text'].apply(lambda x: createEmojiList(x, emoji_list))
print(len(emoji_list))
emoji_list

1645


['grinning_face_with_sweat',
 'crossed_fingers',
 'hole',
 'person_shrugging',
 'rolling_on_the_floor_laughing',
 'baseball',
 'enraged_face',
 'person_facepalming',
 'light_skin_tone:',
 'male_sign',
 'face_with_symbols_on_mouth',
 'red_heart',
 'pizza',
 'winking_face',
 'face_vomiting',
 'grinning_face_with_big_eyes',
 'beating_heart',
 'crystal_ball',
 'growing_heart',
 'smiling_face_with_heart-eyes',
 'star-struck',
 'female_sign',
 'smiling_face_with_tear',
 'face_screaming_in_fear',
 'thumbs_up',
 'face_savoring_food',
 'poultry_leg',
 'sad_but_relieved_face',
 'thinking_face',
 'face_with_tears_of_joy',
 'prohibited',
 'face_with_rolling_eyes',
 'clown_face',
 'party_popper',
 'clapping_hands',
 'medium-light_skin_tone:',
 'zany_face',
 'middle_finger',
 'pile_of_poo',
 'victory_hand',
 'beaming_face_with_smiling_eyes',
 'smiling_face_with_open_hands',
 'face_with_crossed-out_eyes',
 'dizzy',
 'grinning_face',
 'smirking_face',
 'ewe',
 'winking_face_with_tongue',
 'squinting_f

In [13]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_22708\1801815014.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title', 'messages_clean', 'emojis', 'emojis_text'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')


***
## Topics
***

In [3]:
df_merge = pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative,emoji_count
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,Faux jamais temps réélu parti crève faim Jamai...,[],[],0.077610,0.130838,0.791552,0
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100,1


In [5]:
df = df_merge.head(1000).copy()

text = df['message_comments'][0]
text

'Faux, ce ne sera jamais le temps pour lui car il ne sera pas réélu et que son parti crève de faim. Jamais ils ne seront prêts pour une campagne électorale, leurs coffres étant désespérément vides. C\'est la seule et unique raison pour laquelle il s\'est transformé en "lèche-cul" du parti Libéral.'

In [8]:
from transformers import pipeline

classifier = pipeline(model="facebook/bart-large-mnli")

candidate_labels=['Politique', 'Sport', 'Environnement']

results = classifier(text, candidate_labels)


# final_result

In [ ]:
from transformers import pipeline
classifier = pipeline(model="facebook/bart-large-mnli")


candidate_labels=[
    "Changement_climatique",
    "Géopolitique",
    "COVID-19",
    "Technologie",
    "Économie",
    "Justice_sociale",
    "Migration",
    "Éducation_en_ligne",
    "Santé_mentale",
    "Démographie"
]
result_list = []

messages_list = df['message_comments'].tolist()

for message in messages_list:
    results = classifier(text, candidate_labels)
    final_result = max(zip(results['scores'], results['labels']))[1]
    result_list.append(final_result)


result_list

In [13]:
from transformers import pipeline

classifier = pipeline(model="facebook/bart-large-mnli")
candidate_labels=[
    "Changement_climatique",
    "Géopolitique",
    "COVID-19",
    "Technologie",
    "Économie",
    "Justice_sociale",
    "Migration",
    "Éducation_en_ligne",
    "Santé_mentale",
    "Démographie"
]

result_list = [max(zip(classifier(message, candidate_labels)['scores'], candidate_labels))[1] for message in df['message_comments'].tolist()]


In [10]:
result_list

['Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Politique',
 'Poli

In [11]:
df['message_comments'].tolist()

['Faux, ce ne sera jamais le temps pour lui car il ne sera pas réélu et que son parti crève de faim. Jamais ils ne seront prêts pour une campagne électorale, leurs coffres étant désespérément vides. C\'est la seule et unique raison pour laquelle il s\'est transformé en "lèche-cul" du parti Libéral.',
 'Mon dieu!! On nous prend sérieusement pour des idiots 😅',
 'Denise Morais ..Oui et avec raison ! (Car "nous ",  étant la majorité tyrannique des citoyens vendus et  "dociles"  à la   servitude volontaire , sauf exceptions . )  On l\'a expérimenté solide avec  McKinsey et les incongruences des politiciens .) ..."Mentez, mentez , il en restera toujours quelque chose" ... Au Québec ,c\'est gagnant car les gens sont formatés solide !',
 'Gros show. Mais pendant la plandémie chinoise tlm fermait bien sa gueule..',
 'On apellait ça chinada pis les vaxxinés comprenaient rien tu vois ben que yont pas de cerveau',
 'Chinadaaaaaa',
 "Trop drôle,  crevé les caves , vous allez voir que cyclistes en 

***
## Analyse de sentiments
***

In [17]:
df_merge = pd.read_hdf(cwd + '\\dataframe\\' + 'df_merge.h5')
df_multi_student = pd.read_hdf(cwd + '\\dataframe\\' + 'sentiement_model_results.h5')

df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...,[],[]
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat]


In [137]:
from transformers import pipeline
df = df_merge[['message_comments']].copy()
list_to_process = df['message_comments'].tolist()

In [139]:
# Model
distilled_student_sentiment_classifier = pipeline(model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", return_all_scores=True, truncation=True)
result_multi_student = distilled_student_sentiment_classifier(list_to_process)

# Change to df
data = [[d["score"] for d in result_multi_student[i]] for i in range(len(result_multi_student))]
col_names = ['positive', 'neutral', 'negative']
df_multi_student = pd.DataFrame(data, columns=col_names)

In [140]:
# Save results
df_multi_student.to_hdf(cwd + '\\dataframe\\' + 'sentiement_model_results.h5', key='s')

In [124]:
# Model
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_pipeline_multilingual = pipeline('sentiment-analysis', model=model_name, tokenizer=model_name)
result_multi = sentiment_pipeline_multilingual(list_to_process, return_all_scores=True)

# Change to df
data = [[d["score"] for d in result_multi[i]] for i in range(len(result_multi))]
col_names = ['1stars', '2stars', '3stars', '4stars', '5stars']
df_multi = pd.DataFrame(data, columns=col_names)

In [141]:
# df_test = pd.concat([df, df_multi, df_multi_student], axis='columns')
# df_test = pd.concat([df, df_multi_student], axis='columns')

# df_test

,message_comments,positive,neutral,negative
0,"Faux, ce ne sera jamais le temps pour lui car ...",0.077610,0.130838,0.791552
1,Mon dieu!! On nous prend sérieusement pour des...,0.697028,0.047872,0.255100
2,"Denise Morais ..Oui et avec raison ! (Car ""nou...",0.243069,0.089332,0.667599
3,Gros show. Mais pendant la plandémie chinoise ...,0.685404,0.125043,0.189553
4,On apellait ça chinada pis les vaxxinés compre...,0.158446,0.242306,0.599248
...,...,...,...,...
935693,Marcel Routhier si tu n'as pas entendu parler ...,0.066566,0.255113,0.678320
935694,"Gaston Crete en 2002 c,était Rémy Trudel et 20...",0.451840,0.213279,0.334880
935695,"Gaston Crete et c,est là Marois qui a parti le...",0.801224,0.069705,0.129071
935696,Ils commencent à se fâcher le monde ne veulen...,0.187257,0.304669,0.508074


In [20]:
df_merge = pd.concat([df_merge, df_multi_student], axis='columns')
df_merge.head(2)

,IDENTITY_ATTACK,INSULT,PROFANITY,SEVERE_TOXICITY,THREAT,TOXICITY,comment_count,like_count,message_comments,mainTopic,message_posts,secondTopic,shares,title,time_difference,year,month,weekday,IDENTITY_ATTACK_cube,INSULT_cube,PROFANITY_cube,SEVERE_TOXICITY_cube,THREAT_cube,TOXICITY_cube,comment_count_cube,like_count_cube,shares_cube,messages_clean,emojis,emojis_text,positive,neutral,negative
0,0.102216,0.651296,0.664565,0.350583,0.029933,0.588517,0,0,"Faux, ce ne sera jamais le temps pour lui car ...",composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,3.0,2023,6,4,0.467562,0.866814,0.872661,0.705121,0.310492,0.838017,0.0,0.0,0.0,faux jamais temps réélu parti crève faim jamai...,[],[],0.077610,0.130838,0.791552
1,0.005476,0.167515,0.255499,0.006981,0.005647,0.176317,1,1,Mon dieu!! On nous prend sérieusement pour des...,composer-preview,HÉLÈNE BUZZETTI / Jagmeet Singh a soutenu que ...,7FJ4TUHKEFEXFIZI6DY2WAQE4E,0,Les incohérences de l’ingérence,18.0,2023,6,4,0.176257,0.551253,0.634546,0.191119,0.178081,0.560745,1.0,1.0,0.0,dieu prend sérieusement idiots 😅,[😅],[grinning_face_with_sweat],0.697028,0.047872,0.255100


In [21]:
df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')

C:\Users\Charles_tour\AppData\Local\Temp\ipykernel_20500\1801815014.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['message_comments', 'mainTopic', 'message_posts', 'secondTopic',
       'title', 'messages_clean', 'emojis', 'emojis_text'],
      dtype='object')]

  df_merge.to_hdf(cwd + '\\dataframe\\' + 'df_merge.h5', key='s')
